In [1]:
# 1. Read the ref-dist-mat into numpy for array
# 2. Replace array names with annotations
# 3. Double for loop to calculate F1 and F2
# 4. Use F measuremnts to calculated M and FST measures

In [20]:
import numpy
from copy import deepcopy

In [3]:
dist_mat = numpy.genfromtxt("/home/echarvel/Desktop/calab_data/code_bases/Skmer-2/fst_test_data/test_matrix.txt", dtype="str")
annots = numpy.genfromtxt("/home/echarvel/Desktop/calab_data/code_bases/Skmer-2/fst_test_data/test_annots.txt", dtype="str")

for line in annots:
    x=numpy.where(dist_mat == line[0])
    for index in x: 
        try:
            dist_mat[index[0]][index[1]] = line[1]
        except: pass

In [4]:
dist_dict = dict()

for num1 in range(1, len(dist_mat[0])):
    for num2 in range(1, len(dist_mat[0])):
        if num1 != num2: 
            name1=dist_mat[0][num1]
            name2=dist_mat[0][num2]
            if name1 not in dist_dict:
                dist_dict[name1] = dict()

            if name2 not in dist_dict[name1]:
                dist_dict[name1][name2] = [float(dist_mat[num1, num2])]
            else: 
                dist_dict[name1][name2].append(float(dist_mat[num1, num2]))

print(dist_dict)

{'BOAR1': {'BOAR1': [0.01, 0.1], 'PIG2': [0.15, 0.1, 0.8, 0.9]}, 'PIG2': {'BOAR1': [0.15, 0.8, 0.1, 0.9], 'PIG2': [0.02, 0.02]}}


In [5]:
def calculate_WC_FST(F1, F2, M):
    numerator_wc_fst = F1 + F2
    denominator_wc_fst = F1 + F2 + 2 * (1 / (M + 1)) * (M * (1 - F1) + (1 - F2))
    WC_FST = numerator_wc_fst / denominator_wc_fst
    return WC_FST

def calculate_Nei_FST(F1, F2):
    numerator_nei_fst = F1 + F2
    denominator_nei_fst = 2 - ((F1 + F2) / 2)
    Nei_FST = numerator_nei_fst / denominator_nei_fst
    return Nei_FST

def calculate_Hudson_FST(F1, F2):
    Hudson_FST = (F1 + F2) / 2
    return Hudson_FST

In [21]:
wh_fst_mtrx = [[0 for x in range(0, len(dist_dict)+1)] for x in range(0, len(dist_dict)+1)]
wh_fst_mtrx[0][0]="sample"

key_list = list(dist_dict.keys())
for x in range(0, len(key_list)):
    wh_fst_mtrx[0][x+1] = wh_fst_mtrx[x+1][0] = (key_list[x])

wc_fst_mtrx = deepcopy(wh_fst_mtrx)
nei_fst_mtrx = deepcopy(wh_fst_mtrx)
hud_fst_mtrx = deepcopy(wh_fst_mtrx)

In [26]:
for key1, index1 in zip(key_list, range(len(key_list))):
     for key2, index2 in zip(key_list, range(len(key_list))):
        if key1 == key2:
            pass
        else:
            # calculating within-clade average distance
            dist_array_k1_k1 = dist_dict[key1][key1]
            within_k1 = (sum(dist_array_k1_k1)/len(dist_array_k1_k1))

            dist_array_k2_k2 = dist_dict[key2][key2]
            within_k2 = sum(dist_array_k2_k2)/len(dist_array_k2_k2)

            # calculating across-clade average distance
            dist_array_k1_k2 = dist_dict[key1][key2]
            across_k1_k2 = (sum(dist_array_k1_k2)/len(dist_array_k1_k2))

            F1 = within_k1/across_k1_k2
            F2 = within_k2/across_k1_k2

            if len(dist_array_k1_k1) <= len(dist_array_k2_k2):
                M=len(dist_array_k1_k1)/len(dist_array_k2_k2)
            else:
                M=len(dist_array_k2_k2)/len(dist_array_k1_k1)

            wh_fst_mtrx[index1+1][index2+1] = round((F1 + F2)/2, 5)
            wc_fst_mtrx[index1+1][index2+1] = round(calculate_WC_FST(F1, F2, M), 5)
            nei_fst_mtrx[index1+1][index2+1] = round(calculate_Nei_FST(F1, F2), 5)
            hud_fst_mtrx[index1+1][index2+1] =  round(calculate_Hudson_FST(F1, F2), 5)


In [31]:
numpy.savetxt("./wh_fst_mat.txt", numpy.matrix(wh_fst_mtrx), fmt="%s", delimiter="\t")
